# Computer Vision: YOLO for identifying Formula 1 Teams

This notebook contains the essential code for training a YOLOv8 model to detect and classify Formula 1 teams from images and videos.

In [1]:
# Imports
import os
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
from ultralytics import YOLO
import cv2
from torch.optim import Adam
import numpy as np
import random
import shutil
import yaml
from collections import Counter
import time

---

## 1. Path Configuration

Define the paths to your training images and dataset

In [2]:
# Path constants - update these to match your file structure
TRAIN_IMAGE = r"f1-dataset/train/images/1-2023-Brazilian-GP-FP1-20_jpg.rf.2f3d25867c8e1661f6070ae5a84c6dd4.jpg"
IMAGE_FOLDER = r"f1-dataset/train/images"
DATA_YAML = r"f1-dataset/data.yaml"
PROJECT_DIR = r"yolo-files/runs"
WEIGHTS_DIR = r"weights"

---

## 2. Check PyTorch Version and GPU Availability

In [3]:
# Check PyTorch version and CUDA availability
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")

PyTorch Version: 2.5.1+cu124
CUDA Available: True
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU


---

## 3. Training Configuration

Define the parameters for training the model

In [4]:
# Device and training configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_CLASSES = 10  # Number of F1 teams
LEARNING_RATE = 0.0005
BATCH_SIZE = 8  # Adjust based on GPU memory
FINE_TUNED_BATCH_SIZE = 6
NUM_WORKERS = 8  # Increase if you have more CPU cores
NUM_EPOCHS = 50  # Or more, depending on performance
FINE_TUNED_EPOCHS = 30
AUGMENTATION = True  # Data augmentation flag

## 3.1 Image Verification

Verify that the training images can be loaded correctly

In [5]:
# Verify a sample image loads correctly
train_image = cv2.imread(TRAIN_IMAGE)
if train_image is None:
    print("Error loading the image")
else:
    print("Image loaded correctly: ", train_image.shape)

Image loaded correctly:  (640, 640, 3)


## 3.2 Check Image Sizes

Ensure all images have consistent dimensions

In [6]:
# Define the image folder
image_folder = IMAGE_FOLDER
# List to store image sizes
sizes = set()

# Check the size of each image
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    img = cv2.imread(image_path)

    if img is not None:
        img_size = img.shape  # (height, width, channels)
        sizes.add(img_size)
    else:
        print(f"Error loading image: {image_name}")
        
# Check if all images have the same size
if len(sizes) == 1:
    print("All images have the same size.")
else:
    print("There are images with different sizes. The sizes found are:")
    for size in sizes:
        print(size)

All images have the same size.


---

## 4. Utility Functions

Functions for model setup, training, evaluation and testing

In [7]:
# For reproducibility
def set_seed(seed=42):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    print(f"Random seed set to {seed}")

In [8]:
def load_model(weights_path: str = None, default_model: str = "yolo12m.pt"):
    """Load a YOLO model from weights or use a named pretrained model."""
    if weights_path and os.path.exists(weights_path):
        model = YOLO(weights_path)
        print(f"Loaded model from {weights_path}")
    else:
        model = YOLO(default_model)
        print(f"Loaded pretrained {default_model} model")
    return model

In [9]:
# Update data.yaml with class weights
def update_data_yaml(yaml_path, class_weights=None):
    """Update data.yaml with custom class weights"""
    # Read existing data.yaml
    with open(yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    # Create backup if none exists
    backup_file = yaml_path + '.backup'
    if not os.path.exists(backup_file):
        shutil.copy(yaml_path, backup_file)
        print(f"Backup created at {backup_file}")
    
    # Get class names
    class_names = data_config.get('names', [])
    
    # Update with class weights if provided
    if class_weights:
        data_config['class_weights'] = class_weights
        
        # Save the updated file
        with open(yaml_path, 'w') as f:
            yaml.dump(data_config, f, default_flow_style=False)
        
        print(f"Updated {yaml_path} with custom class weights:")
        for i, name in enumerate(class_names):
            weight = class_weights.get(i, 1.0)
            print(f"  - {name}: {weight}")
    
    return data_config

---

## 5. Model Training Function

Core function to train the YOLO model with optimized hyperparameters

In [10]:
# Train a model with given parameters
def train_model(model, data_yaml_path, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, 
                lr=LEARNING_RATE, project_path=PROJECT_DIR, run_name='train'):
    """Train a YOLO model with specified parameters"""
    
    print(f"\n=== TRAINING CONFIGURATION ===")
    print(f"Epochs: {epochs}")
    print(f"Batch size: {batch_size}")
    print(f"Learning rate: {lr}")
    print(f"Project path: {project_path}")
    print(f"Run name: {run_name}")
    print("===================================\n")
    
    # Check if directories exist, create if not
    os.makedirs(project_path, exist_ok=True)
    
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        batch=batch_size,
        imgsz=640,  # Image size
        optimizer="AdamW",
        lr0=lr,
        lrf=0.01,  # Final LR factor
        cos_lr=True,  # Use cosine learning rate schedule
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3,
        
        # Loss weightings
        box=7.5,  # Box loss weight
        cls=7.0,  # Class loss weight
        dfl=1.5,  # Distribution focal loss weight
        
        # Data augmentation 
        mosaic=0.8,        # Mosaic augmentation
        mixup=0.2,         # Mixup augmentation
        copy_paste=0.1,    # Copy-paste augmentation
        degrees=15.0,      # Rotation augmentation
        translate=0.2,     # Translation augmentation
        scale=0.5,         # Scale augmentation
        shear=1.0,         # Shear augmentation
        fliplr=0.5,        # Horizontal flip probability
        flipud=0.05,       # Vertical flip probability
        hsv_h=0.015,       # Hue augmentation
        hsv_s=0.7,         # Saturation augmentation
        hsv_v=0.5,         # Value/brightness augmentation
        
        # Early stopping
        patience=15,
        
        # Device
        device=DEVICE,
        
        # Pretrained weights
        pretrained=True,
        
        # Save settings
        project=project_path,
        name=run_name,
        save_period=5,      # Save checkpoint every 5 epochs
        
        # Additional options
        verbose=True,       # Print training statistics
        val=True,           # Run validation during training
        plots=True,         # Create training plots
    )
    
    print(f"Training completed for {epochs} epochs")
    return results

---

## 6. Fine-Tuning Function

For fine-tuning an already trained model for better performance on specific classes

In [11]:
def fine_tune_model(model, data_yaml_path, epochs=12, batch_size=6,  # increased to utilize ~8GB VRAM
                   lr=LEARNING_RATE/5, project_path=PROJECT_DIR, run_name='fine_tune_v12m_4060'):
    """Fine-tune a YOLOv12-medium on a single NVIDIA 4060 (8GB VRAM)"""
    print(f"\n=== FINE-TUNING CONFIGURATION (4060 - 8GB) ===")
    print(f"Epochs: {epochs}")
    print(f"Batch size: {batch_size}")
    print(f"Image size: 640x640")
    print(f"Learning rate: {lr}")
    print(f"Project path: {project_path}")
    print(f"Run name: {run_name}")
    print("============================================\n")

    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        batch=batch_size,
        imgsz=640,
        optimizer="AdamW",
        lr0=lr,
        lrf=0.01,
        cos_lr=True,
        momentum=0.937,
        weight_decay=0.001,
        warmup_epochs=3,
        # Loss weightings
        box=5.5,
        cls=15.0,
        dfl=1.5,
        # Moderate augmentation
        mosaic=0.5,
        mixup=0.1,
        copy_paste=0.05,
        degrees=5.0,
        translate=0.05,
        scale=0.3,
        shear=0.2,
        fliplr=0.5,
        flipud=0.0,
        hsv_h=0.01,
        hsv_s=0.3,
        hsv_v=0.2,
        # Early stopping
        patience=20,
        # Device
        device=DEVICE,
        # Memory optimization
        cache=False,
        amp=True,
        # Pretrained weights
        pretrained=True,
        project=project_path,
        name=run_name,
        save_period=2,
        verbose=True,           # enable verbose logging
        val=True,
        plots=True,             # re-enable plots for monitoring
    )
    print(f"Fine-tuning completed for {epochs} epochs on 4060 (8GB)")
    return results


---

## 7. Model Evaluation and Saving Functions

In [12]:
def evaluate_model(model):
    metrics = model.val()
    print(f"mAP50: {metrics.box.map50:.4f}")
    print(f"mAP50-95: {metrics.box.map:.4f}")
    print(f"Precision: {metrics.box.mp:.4f}")
    print(f"Recall: {metrics.box.mr:.4f}")
    return metrics


In [13]:
# Save a model
def save_model(model, path):
    """Save a YOLO model to specified path"""
    # Ensure directory exists
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    model.save(path)
    print(f"Model saved to {path}")

In [14]:
# Test a model on images
def test_model(model, test_images_path, save_results=True):
    """Test a YOLO model on test images"""
    print(f"Testing model on images in {test_images_path}...")
    
    results = model.predict(
        source=test_images_path,
        conf=0.25,           # Confidence threshold
        iou=0.45,            # IoU threshold
        max_det=20,          # Maximum detections per image
        save=save_results    # Save results to disk
    )
    
    print(f"Testing completed")
    return results

---

## 8. Example workflow: Class Weight Balancing

Function to update class weights based on model performance

In [15]:
# Example: Update class weights based on validation results
def balance_class_weights(data_yaml_path, class_performance):
    """Update class weights based on model performance
    
    Args:
        data_yaml_path: Path to data.yaml file
        class_performance: Dictionary with class names as keys and mAP values as values
    """
    # Read existing data.yaml
    with open(data_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    # Get class names
    class_names = data_config.get('names', [])
    
    # Create class weights - higher weight for lower performing classes
    class_weights = {}
    for i, name in enumerate(class_names):
        # Default weight is 1.0
        weight = 1.0
        
        # If we have performance data for this class
        if name in class_performance:
            map_value = class_performance[name]
            
            # Inverse relationship: lower mAP gets higher weight
            if map_value < 0.5:
                weight = 2.0  # Very poor performance, high weight
            elif map_value < 0.7:
                weight = 1.5  # Medium performance, medium weight
            elif map_value < 0.9:
                weight = 1.0  # Good performance, normal weight
            else:
                weight = 0.8  # Excellent performance, slightly lower weight
        
        class_weights[i] = weight
    
    # Update data.yaml with new weights
    return update_data_yaml(data_yaml_path, class_weights)

In [16]:
# Example: Load and train a model
model = load_model()
train_results = train_model(model, DATA_YAML, epochs=40, run_name='yolo12_m_training')

Loaded pretrained yolo12m.pt model

=== TRAINING CONFIGURATION ===
Epochs: 40
Batch size: 8
Learning rate: 0.0005
Project path: yolo-files/runs
Run name: yolo12_m_training

engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=7.0, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=f1-dataset/data.yaml, degrees=15.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.05, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.5, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolo12m.pt, momentum=0.937, mosaic=0.8, multi_scale=False, name=yolo12_m_training, nbs=64, nms=False, opset=None, optimize=False, opt

train: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\train\labels.cache... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 420.2126.2 MB/s, size: 54.2 KB)


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to yolo-files\runs\yolo12_m_training\labels.jpg... 
optimizer: AdamW(lr=0.0005, momentum=0.937) with parameter groups 123 weight(decay=0.0), 130 weight(decay=0.0005), 129 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo-files\runs\yolo12_m_training
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      40/40         6G     0.6823      5.291      1.214          7        640: 100%|██████████| 313/313 [01:36<00:00,  3.26it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.60it/s]

                   all         20         29      0.914      0.786      0.919      0.755



40 epochs completed in 1.084 hours.
Optimizer stripped from yolo-files\runs\yolo12_m_training\weights\last.pt, 40.8MB
Optimizer stripped from yolo-files\runs\yolo12_m_training\weights\best.pt, 40.8MB

Validating yolo-files\runs\yolo12_m_training\weights\best.pt...
Ultralytics 8.3.137  Python-3.10.17 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLOv12m summary (fused): 169 layers, 20,112,622 parameters, 0 gradients, 67.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.64it/s]


                   all         20         29      0.778      0.894      0.942      0.758
           Kick Sauber          4          6          1      0.641      0.866      0.718
          Racing Bulls          1          1      0.331          1      0.995      0.697
                Alpine          4          5          1      0.988      0.995      0.804
               Ferarri          3          4      0.887          1      0.995       0.86
                  Haas          1          1      0.652          1      0.995      0.796
               Mclaren          5          6          1      0.523      0.697      0.589
              Mercedes          3          3      0.834          1      0.995      0.895
              Williams          2          3      0.522          1      0.995      0.704
Speed: 0.2ms preprocess, 10.8ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to yolo-files\runs\yolo12_m_training
Training completed for 40 epochs


In [17]:
# Example: Evaluate a model
metrics = evaluate_model(model)

Ultralytics 8.3.137  Python-3.10.17 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLOv12m summary (fused): 169 layers, 20,112,622 parameters, 0 gradients, 67.2 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 568.3175.4 MB/s, size: 54.5 KB)


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.27it/s]


                   all         20         29      0.778      0.894      0.942      0.755
           Kick Sauber          4          6          1      0.641      0.866      0.718
          Racing Bulls          1          1       0.33          1      0.995      0.697
                Alpine          4          5          1      0.988      0.995      0.784
               Ferarri          3          4      0.885          1      0.995       0.86
                  Haas          1          1      0.651          1      0.995      0.796
               Mclaren          5          6          1      0.525      0.697      0.589
              Mercedes          3          3      0.834          1      0.995      0.895
              Williams          2          3      0.522          1      0.995      0.704
Speed: 1.2ms preprocess, 33.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to yolo-files\runs\yolo12_m_training2
mAP50: 0.9417
mAP50-95: 0.7554
Precision: 0.7776
Recall: 0.8942


In [18]:
save_model(model, "weights/yolo_medium_detection.pt")

Model saved to weights/yolo_medium_detection.pt


In [19]:
# Example: Fine-tune a model for specific classes
# Custom class weights for problematic classes

model = load_model(weights_path="weights/yolo_medium_detection.pt")
class_weights = {
    0: 1.2,    # Kick Sauber - slight increase 
    1: 1.0,    # Racing Bulls - balanced
    2: 1.3,    # Alpine - increase to improve precision
    3: 0.9,    # Ferrari - slight reduction
    4: 0.8,    # Haas - reduction to balance
    5: 2.5,    # McLaren - significant increase to improve precision
    6: 1.5,    # Mercedes - moderate increase
    7: 1.0     # Williams - maintain
}
update_data_yaml(DATA_YAML, class_weights)


Loaded model from weights/yolo_medium_detection.pt
Updated f1-dataset/data.yaml with custom class weights:
  - Kick Sauber: 1.2
  - Racing Bulls: 1.0
  - Alpine: 1.3
  - Aston Martin: 0.9
  - Ferarri: 0.8
  - Haas: 2.5
  - Mclaren: 1.5
  - Mercedes: 1.0
  - Red Bull: 1.0
  - Williams: 1.0


{'augment': True,
 'class_weights': {0: 1.2,
  1: 1.0,
  2: 1.3,
  3: 0.9,
  4: 0.8,
  5: 2.5,
  6: 1.5,
  7: 1.0},
 'names': ['Kick Sauber',
  'Racing Bulls',
  'Alpine',
  'Aston Martin',
  'Ferarri',
  'Haas',
  'Mclaren',
  'Mercedes',
  'Red Bull',
  'Williams'],
 'nc': 10,
 'roboflow': {'license': 'CC BY 4.0',
  'project': 'f1-car-2023',
  'url': 'https://universe.roboflow.com/f1detectionaiclass/f1-car-2023/dataset/1',
  'version': 1,
  'workspace': 'f1detectionaiclass'},
 'test': 'C:/Users/victo/Desktop/Documents/Tercer año/Segundo Cuatrimestre/Finales/f1-strategy/f1-dataset/test/images',
 'train': 'C:/Users/victo/Desktop/Documents/Tercer año/Segundo Cuatrimestre/Finales/f1-strategy/f1-dataset/train/images',
 'val': 'C:/Users/victo/Desktop/Documents/Tercer año/Segundo Cuatrimestre/Finales/f1-strategy/f1-dataset/valid/images'}

In [20]:
fine_tune_results = fine_tune_model(model, DATA_YAML, epochs=15, run_name='class_specific_tuning')


=== FINE-TUNING CONFIGURATION (4060 - 8GB) ===
Epochs: 15
Batch size: 6
Image size: 640x640
Learning rate: 0.0001
Project path: yolo-files/runs
Run name: class_specific_tuning

engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=5.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=15.0, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=f1-dataset/data.yaml, degrees=5.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.3, hsv_v=0.2, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=weights/yolo_medium_detection.pt, momentum=0.937, mosaic=0.5, multi_scale=False, name=class_specific_tuning, nbs=64, nms=False, o

train: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\train\labels.cache... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 768.2230.8 MB/s, size: 54.2 KB)


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to yolo-files\runs\class_specific_tuning\labels.jpg... 
optimizer: AdamW(lr=0.0001, momentum=0.937) with parameter groups 123 weight(decay=0.0), 130 weight(decay=0.00103125), 129 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo-files\runs\class_specific_tuning
Starting training for 15 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      15/15      4.52G     0.4501      9.947      1.139          6        640: 100%|██████████| 417/417 [01:37<00:00,  4.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.60it/s]

                   all         20         29      0.922      0.774      0.925      0.753



15 epochs completed in 0.419 hours.
Optimizer stripped from yolo-files\runs\class_specific_tuning\weights\last.pt, 40.8MB
Optimizer stripped from yolo-files\runs\class_specific_tuning\weights\best.pt, 40.8MB

Validating yolo-files\runs\class_specific_tuning\weights\best.pt...
Ultralytics 8.3.137  Python-3.10.17 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLOv12m summary (fused): 169 layers, 20,112,622 parameters, 0 gradients, 67.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.62it/s]


                   all         20         29      0.925      0.771       0.94      0.784
           Kick Sauber          4          6          1      0.526      0.809      0.642
          Racing Bulls          1          1      0.848          1      0.995      0.796
                Alpine          4          5          1      0.447      0.962      0.851
               Ferarri          3          4          1      0.819      0.995      0.931
                  Haas          1          1      0.881          1      0.995      0.895
               Mclaren          5          6          1      0.378      0.859      0.709
              Mercedes          3          3      0.975          1      0.995      0.796
              Williams          2          3      0.698          1      0.913      0.651
Speed: 0.4ms preprocess, 10.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to yolo-files\runs\class_specific_tuning
Fine-tuning completed for 15 epochs on 4060 (8GB)


In [26]:
metrics = evaluate_model(model)

Ultralytics 8.3.137  Python-3.10.17 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLOv12m summary (fused): 169 layers, 20,112,622 parameters, 0 gradients, 67.2 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 628.4168.5 MB/s, size: 54.5 KB)


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.53it/s]


                   all         20         29      0.925      0.771       0.94      0.781
           Kick Sauber          4          6          1      0.526      0.809      0.642
          Racing Bulls          1          1      0.848          1      0.995      0.796
                Alpine          4          5          1      0.447      0.962      0.851
               Ferarri          3          4          1      0.819      0.995       0.91
                  Haas          1          1      0.881          1      0.995      0.895
               Mclaren          5          6          1      0.378      0.859      0.709
              Mercedes          3          3      0.975          1      0.995      0.796
              Williams          2          3      0.698          1      0.913      0.651
Speed: 1.6ms preprocess, 36.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to yolo-files\runs\class_specific_tuning2
mAP50: 0.9403
mAP50-95: 0.7813
Precision: 0.9252
Recall: 0.7

In [24]:
fine_tune_model = save_model(model, "weights/fine_tuned.pt")

Model saved to weights/fine_tuned.pt
